In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [8]:
import zipfile
# 1. 압축 해제
zip_path = "/content/drive/MyDrive/pilly/images_324.zip"
extract_dir = "dataset_raw"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [9]:
# prepare_data.py
import os

import shutil
import random
from sklearn.model_selection import train_test_split

# --- 설정 변수 정의 ---
zip_path = "images_216.zip" # 당신의 ZIP 파일 경로
extract_dir = "dataset_raw" # 압축 해제될 폴더

# 압축 해제 후 원본 이미지가 있는 디렉토리
# (가장 중요한 부분: dataset_raw 바로 밑에 클래스 폴더가 있다면 extract_dir,
#  dataset_raw/images 처럼 images 폴더 안에 클래스가 있다면 os.path.join(extract_dir, 'images') 로 변경)
original_images_base_dir = extract_dir

# 데이터 분할 후 저장될 디렉토리 (이 폴더가 최종적으로 'data' 인자에 사용될 것입니다)
split_data_base_dir = './data_split_yolov8'
train_dir = os.path.join(split_data_base_dir, 'train')
val_dir = os.path.join(split_data_base_dir, 'val')

# 데이터 분할 비율
val_split_ratio = 0.2

# --- 압축 해제 함수 ---
def extract_zip(zip_path, extract_to):
    if not os.path.exists(extract_to) or not os.listdir(extract_to):
        print(f"압축 해제 중: {zip_path} -> {extract_to}")
        os.makedirs(extract_to, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print("압축 해제 완료.")
    else:
        print(f"'{extract_to}' 디렉토리가 이미 존재하며 비어있지 않습니다. 압축 해제를 건너뜝니다.")

# --- 데이터 분할 함수 ---
def split_classification_data(src_dir, train_dest, val_dest, val_ratio):
    print(f"\n--- 데이터 분할 시작: train_ratio={1-val_ratio:.2f}, val_ratio={val_ratio:.2f} ---")

    if os.path.exists(train_dest): shutil.rmtree(train_dest)
    if os.path.exists(val_dest): shutil.rmtree(val_dest)
    os.makedirs(train_dest, exist_ok=True)
    os.makedirs(val_dest, exist_ok=True)

    classes = sorted([d for d in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, d))])
    if not classes:
        raise ValueError(f"'{src_dir}'에서 클래스 폴더를 찾을 수 없습니다. 원본 데이터 구조를 확인해주세요.")

    print(f"발견된 클래스: {classes}")

    for cls in classes:
        cls_path = os.path.join(src_dir, cls)
        images = [os.path.join(cls_path, img) for img in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, img))]
        train_imgs, val_imgs = train_test_split(images, test_size=val_ratio, random_state=42)

        os.makedirs(os.path.join(train_dest, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dest, cls), exist_ok=True)

        for img in train_imgs: shutil.copy(img, os.path.join(train_dest, cls, os.path.basename(img)))
        for img in val_imgs: shutil.copy(img, os.path.join(val_dest, cls, os.path.basename(img)))

        print(f"  클래스 '{cls}': 훈련 이미지 {len(train_imgs)}개, 검증 이미지 {len(val_imgs)}개 분할 완료.")

    print("--- 데이터 분할 완료 ---")


if __name__ == "__main__":
    if os.path.exists(split_data_base_dir):
        print(f"기존 '{split_data_base_dir}' 디렉토리 삭제 중...")
        shutil.rmtree(split_data_base_dir)
        print("삭제 완료.")

    extract_zip(zip_path, extract_dir)

    try:
        split_classification_data(original_images_base_dir, train_dir, val_dir, val_split_ratio)
    except ValueError as e:
        print(f"오류: {e}")
        print("데이터셋 구조가 예상과 다릅니다. 'original_images_base_dir' 경로를 확인하거나 ZIP 파일 내부 구조를 검토해주세요.")
        exit()

    print(f"\n데이터 준비 완료. 학습을 위해 '{split_data_base_dir}' 폴더를 사용하세요.")

'dataset_raw' 디렉토리가 이미 존재하며 비어있지 않습니다. 압축 해제를 건너뜝니다.

--- 데이터 분할 시작: train_ratio=0.80, val_ratio=0.20 ---
발견된 클래스: ['K-000041', 'K-000135', 'K-000473', 'K-000514', 'K-000551', 'K-000742', 'K-000744', 'K-000798', 'K-000817', 'K-001739', 'K-001768', 'K-002491', 'K-002680', 'K-002881', 'K-003289', 'K-003487', 'K-003883', 'K-004093', 'K-004158', 'K-004451', 'K-004606', 'K-004699', 'K-005227', 'K-005606', 'K-005779', 'K-008973', 'K-010467', 'K-011952', 'K-012724', 'K-013156']
  클래스 'K-000041': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000135': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000473': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000514': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000551': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000742': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000744': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000798': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-000817': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-001739': 훈련 이미지 172개, 검증 이미지 44개 분할 완료.
  클래스 'K-001768': 훈련 이미지 172개, 검증 

In [10]:
!yolo classify train \
  data=./data_split_yolov8 \
  model=yolov8n-cls.pt \
  epochs=50 \
  imgsz=224 \
  batch=32 \
  name=my_pill_cls_run_cli \
  project=cli_classification_project \
   patience=5 \
  device=0 # GPU 사용 (CPU는 'cpu')!

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data_split_yolov8, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=my_pill_cls_run_cli2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretr

In [7]:
# 폴더 삭제
!rm -rf /content/data_split_yolov8

In [11]:
# 결과 저장
import shutil

src = '/content/cli_classification_project/my_pill_cls_run_cli2'  # YOLO 분류 결과 폴더
dst = '/content/drive/MyDrive/pilly'

shutil.copytree(src, dst, dirs_exist_ok=True)
print("✅ 결과가 Google Drive에 저장되었습니다!")


✅ 결과가 Google Drive에 저장되었습니다!
